# Testing Agents in AI Foundry
This notebook creates and runs an agent from AI Foundry

In [5]:
# Install required packages

!pip install requests "semantic-kernel[azure]>=1.34.0"

## Step 2
Create Project client

In [34]:
import asyncio
from datetime import date
import logging
import os
from azure.identity import DefaultAzureCredential, AzureDeveloperCliCredential

from semantic_kernel.agents import (
    AzureAIAgent,
    AzureAIAgentThread,
    AzureAIAgentSettings,
)

from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()

endpoint = os.environ.get("AZURE_AI_FOUNDRY_CONNECTION_STRING")
deployment_name = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION", None)
tenant_id = os.environ.get("AZURE_TENANT_ID", None)

ai_agent_settings = AzureAIAgentSettings(
    endpoint=endpoint,
    model_deployment_name=deployment_name,
    api_version=api_version,
)

creds = AzureDeveloperCliCredential(tenant_id=tenant_id) if os.environ.get("USE_AZURE_DEV_CLI") == "true" else DefaultAzureCredential()
client = AzureAIAgent.create_client(
    credential=creds,
    endpoint=ai_agent_settings.endpoint,
    api_version=ai_agent_settings.api_version,
)

agent_definition = None

# List agents
async for agent in client.agents.list_agents():
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Description: {agent.description}, Deployment Name: {agent.model}")
    if agent.name == "Joe2":
        agent_definition = agent


Agent ID: asst_ToihIlghRnnuk7fU9ASWEfRN, Name: Joe, Description: None, Deployment Name: gpt-35-turbo
Agent ID: asst_cTmTnzQTfPo8yr838947fnhm, Name: Joe, Description: None, Deployment Name: gpt-35-turbo
Agent ID: asst_G6FGdqwjsuXOewfgkmhwD1QM, Name: Agent778, Description: None, Deployment Name: gpt-35-turbo
Agent ID: asst_SkAs2Pj7omRtGzG7SbJ8QpHv, Name: Agent287, Description: None, Deployment Name: gpt-35-turbo
Agent ID: asst_m7B9QtfNGk99HWSchCBJmuQn, Name: Agent484, Description: None, Deployment Name: gpt-35-turbo


### Create Agent

In [35]:
agent_name = "Joe"
agent_instructions = (
    "You are a helpful assistant named Joe. "
    "You are designed to assist with various tasks and answer questions. "
    "Your responses should be concise and informative."
)

agent_definition = agent_definition if agent_definition else await client.agents.create_agent(
    model=ai_agent_settings.model_deployment_name,
    name=agent_name,
    instructions=agent_instructions,
)
agent = AzureAIAgent(
    client=client,
    definition=agent_definition,
)

### Use the agent

In [33]:
input = "How did Abraham Lincoln die?"
thread = AzureAIAgentThread(client=client)

async for agent_response in agent.invoke(messages=input, thread=thread):
    print(f"Agent: {agent_response}")

Agent: Abraham Lincoln was assassinated on April 14, 1865, by John Wilkes Booth, who shot him at Ford's Theatre in Washington, D.C. Lincoln passed away the following day, April 15, 1865.
